Future possible refinements:
 
If we have caliper, then we can limit the spinner flow rate analysis to only those intervals that are in gauge

Look for data where the spinner was stuck and remove these



## 15. Velocity profiles from spinner data

The spinner tool's impeller rotates in the positive direction when the tool is pulled upwards through the well and in the negitive direction when the tool is lowered down into the well. 

If the tool is moving at the same rate as the injected fluid, then the impeller stops spinner (frequency = 0)

***

***

# 16. Import, mudge and check data

## 16.1 Use bespoke functions to import and mudge data

The data munging process in 1-overview.ipynb has been turned into specilised functions located in the functions.py file. We import and use that set of functions in the same way as any other module.


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from ipywidgets import interactive, Layout, FloatSlider
from utilities import* # functions in the utilities.py file

Because the cell below includes all steps required to import and mudge the data (everything from notebook 1), it will take a little while to run. 

In [ ]:
flowrate = read_flowrate(r'Data-FlowRate.xlsx')
pts = read_pts(r'Data-PTS.xlsx')

In [ ]:
pts.shape

## 16.2 Add flowrate values to the pts dataframe

Our fluid velocity analysis requires that we know the pump rate and spinner frequency. There are several ways we could approch this:

1. We could assume that the pump rate was held perfectly steady the defined pump rate and set a single value
2. We could use the actual flowrate data if that is available

As have good quality surface pupmp data, we use the bespoke function below to append the flowrate data to the pts dataframe. As the flowrate data is recorded at a courser time resolution than the pts data. We used linear interpolation to fill the gaps. 

If you are using this workflow on your own data, you need to adjust the column names in the function. This method is documented in bonus-combine-date.ipynb

In [ ]:
pts = append_flowrate_to_pts(flowrate, pts)

In [ ]:
pts.shape

In [ ]:
pts.head(2)

## 16.3 Check the data

It's good practice to check your data after import. 

You can use the Pandas methods listed in Section 2.1.1 (1-intro-and-data.ipynb) to check your data. 

### 16.3.1 Spinner by depth

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(24,8),sharey=True)

ax1.scatter(pts.frequency_hz, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)
ax2.scatter(pts.datetime, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)

ax3 = ax2.twinx()
ax3.plot(flowrate.datetime, flowrate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax1.set_ylim(1000,0)
ax1.set_xlim(-30,30)

ax1.set_ylabel('Depth [m]')
ax1.set_xlabel('Spinner frequency [hz]')

ax2.set_xlabel('Time [hh:mm]')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax3.set_ylabel('Flowrate [t/hr]')

for ax in [ax1, ax2]:
    ax.grid()

### 16.3.1 Spinner by time

As we zoom into the data, we see that the issue of the slow-down

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(28,8))

#ax1.plot(pts.datetime, pts.frequency_hz, c='k', linestyle = '-', linewidth = 1, alpha = 0.3)
ax1.scatter(pts.datetime, pts.frequency_hz, c = pts.timestamp, s = 5, linewidths = 0)
ax2.scatter(pts.datetime, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)

ax3 = ax2.twinx()
ax3.plot(flowrate.datetime, flowrate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax4 = ax1.twinx()
ax4.plot(pts.datetime, pts.depth_m, 
    c='k', linestyle = '-', linewidth = 2, alpha = 0.3, 
    label='Tool depth [m]')
ax4.set_ylim(1000,-1000)
ax4.set_ylabel('Tool depth [m]')

ax1.set_ylim(-30,30)
ax1.set_ylabel('Spinner frequency [hz]')

ax2.set_ylim(1000,0)
ax2.set_ylabel('Tool depth [m/s]')

for ax in [ax1,ax2]:
    ax.set_xlabel('Time [hh:mm]')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax3.set_ylabel('Flowrate [t/hr]')

for ax in [ax1, ax2]:
    ax.grid()

#
# Limit to a time range
#

start_time = pd.to_datetime('2020-12-11 09:30:00')
end_time = pd.to_datetime('2020-12-11 10:30:00')

ax1.set_xlim(start_time,end_time)
ax4.set_ylim(1000,400);

## 16.4 Clean data

Remove data aquired when the tool is stationary or slowing and the data aquired in the casing

To understand this method and how we decided which data to filter, refer to bonus-filter-by-toolspeed.ipynb

In [ ]:
moving_pts = pts[
    (pts.speed_mps > 0.9 ) & (pts.speed_mps < pts.speed_mps.max()) | 
    (pts.speed_mps > pts.speed_mps.min() ) & (pts.speed_mps < -0.9)
    ]

production_shoe = 462.5  

clean_pts = moving_pts[(moving_pts.depth_m < moving_pts.depth_m.max()) & (moving_pts.depth_m > production_shoe)]

clean_pts.describe()

Check the results of the cleaning

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(28,8))

ax1.scatter(clean_pts.datetime, clean_pts.frequency_hz, c = clean_pts.timestamp, s = 5, linewidths = 0)
ax2.scatter(clean_pts.datetime, clean_pts.depth_m, c = clean_pts.timestamp, s = 5, linewidths = 0)

ax3 = ax2.twinx()
ax3.plot(flowrate.datetime, flowrate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax4 = ax1.twinx()
ax4.plot(pts.datetime, pts.depth_m, 
    c='k', linestyle = '-', linewidth = 2, alpha = 0.3, 
    label='Tool depth [m]')
ax4.set_ylim(1000,400)
ax4.set_ylabel('Tool depth [m]')

ax1.set_ylim(-30,30)
ax1.set_ylabel('Spinner frequency [hz]')

ax2.set_ylim(1000,0)
ax2.set_ylabel('Tool depth [m]')

for ax in [ax1,ax2]:
    ax.set_xlabel('Time [hh:mm]')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

ax3.set_ylabel('Flowrate [t/hr]')

for ax in [ax1, ax2]:
    ax.grid()

#
# Limit to a time range
#

start_time = pd.to_datetime('2020-12-11 14:30:00')
end_time = pd.to_datetime('2020-12-11 15:30:00')

ax1.set_xlim(start_time,end_time);

***

## 17. Select data by flowrate

We do this because...

### 17.1 Interactive plot with ipywidgets

Use the sliders on the interactive plot below to... 

In [ ]:

min_timestamp = pts.timestamp.iloc[0]
max_timestamp = pts.timestamp.iloc[-1]

def subselect_plot(start_value, stop_value):
    f,ax = plt.subplots(1,1, figsize = (20,6))
    ax.scatter(clean_pts.timestamp, clean_pts.depth_m,
        c = 'k', s = 1, linewidths = 0, label = 'Tool depth')
    ax1 = ax.twinx()
    ax1.plot(flowrate.timestamp, flowrate.flow_tph, 
        ':', c='k', label='Surface pump flowrate')
    ymin = pts.depth_m.min()
    ymax = pts.depth_m.max() + 100
    ax.vlines(start_value, ymin, ymax, color='tab:green')
    ax.vlines(stop_value, ymin, ymax, color='tab:red')
    ax.set_ylim(pts.depth_m.max() + 100, 0)
    ax.set_xlabel('Timestamp [sec]')
    ax.set_ylabel('Tool depth [m]')
    ax1.set_ylabel('Flowrate [t/hr]')

result = interactive(subselect_plot,
         
         start_value = FloatSlider
         (
             value = (max_timestamp - min_timestamp)/3 + min_timestamp,
             description = 'start',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=False,
             layout = Layout(width='80%'),
             ),
          
          stop_value = FloatSlider
          (
             value = (max_timestamp - min_timestamp)/2 + min_timestamp, 
             description = 'stop',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=False,
             layout = Layout(width='80%')
             )
)

display(result);


In [ ]:
print(
    'start =',result.children[0].value, ' which is', datetime.fromtimestamp(result.children[0].value), 
    '\n stop =', result.children[1].value, ' which is', datetime.fromtimestamp(result.children[1].value),
    )

## 17.2 Record your analysis

As part of making this process repeatable and so that it is easy to come back later to check the analysis, you need to use the range you have selected to manually define objects in the next step.

Copy-paste the timestamps printed above into the cell below. They are now the objects that define the start and stop points for splicing the moving_pts dataframe. 

We could have defined the start and stop objects as the result.childern\[0\].value and result.childern\[1\].value objects. But if we did this, you would lose your work because these ipywidget results change every time the sliders are moved or the notebook is re-run. 

#### First pump flowrate (lowest rate)

Insert results here

#### Second pump flowrate (highest rate)

start = 1607641584.448  which is 2020-12-11 12:06:24.448000 

stop = 1607644334.448  which is 2020-12-11 12:52:14.448000

#### Third pump flowrate (middle rate)

Insert results here


## 17.3 Make a dataframe for one flowrate

We will do the rest of this analysis with only one rate ... but we need to do at least two ... think about notebook structure

In [ ]:

start = 1607641584.448  # paste your start value here
stop = 1607644334.448   # paste your stop value here
name = 'Second pump rate'  # create an informative title for the data

pts_second_rate = clean_pts[
    (clean_pts.datetime > datetime.fromtimestamp(start)) 
    & (clean_pts.datetime < datetime.fromtimestamp(stop))
]

flowrate_second_rate = flowrate[
    (flowrate.datetime > datetime.fromtimestamp(start)) 
    & (flowrate.datetime < datetime.fromtimestamp(stop))
]

Plot the subselected data

In [ ]:
fig, (ax1, ax3) = plt.subplots(1, 2,figsize=(20,6))

ax1.scatter(pts_second_rate.datetime, pts_second_rate.frequency_hz, 
    c = pts_second_rate.timestamp, s = 5, linewidths = 0)

ax2 = ax1.twinx()
ax2.plot(flowrate_second_rate.datetime, flowrate_second_rate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax3.scatter(pts_second_rate.frequency_hz, pts_second_rate.depth_m, 
    c = pts_second_rate.timestamp, s = 5, linewidths = 0)

ax1.set_ylabel('Spinner frequency [hz]')
ax1.set_xlabel('Time [hh:mm]')
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax2.set_ylabel('Pump rate [t/hr]')

ax3.set_xlabel('Spinner frequency [hz]')
ax3.set_ylabel('Depth [m]')

for ax in [ax1,ax3]:
    ax.grid()

for ax in [ax2, ax4]:
    ax.legend()

***

# 18. Cross-plot analysis

Describe the method here and demostration with one meter of data

In [ ]:
interval_top = 700 # shallowest depth in the cross-plot analysis interval
interval_bottom = 701 # deepest depth in the cross-plot analysis interval


selected_data = pts_second_rate[
    (pts_second_rate.depth_m > interval_top ) & (pts_second_rate.depth_m < interval_bottom)
    ]

linear_model = stats.linregress(selected_data.frequency_hz, selected_data.speed_mps)

slope = linear_model[0]
intercept = linear_model[1] # this is the tool speed that matches the fluid velocity
rvalue = linear_model[2] # this is how well the model fits the data and will be a filter

print(rvalue, rvalue**2, intercept) 

The stats.linregress method returns the R value, which is a number between 1 and -1 where:
- -1 indicates that an **increase** in x has an assocated **decrease** in y
- +1 indicates that an **increase** in x has an assocated **increase** in y
- 0 indicates there is **no relationship** between x and y

The $R^2$ value tells us how related x and y are. $R^2$ varies between 0 and 1 and can be thought of how well the model explains the varation. If $R^2 = 0.99$ then the model expalins 99% of the varation in the data, which is a very good fit. While an $R^2 = 0.3$ indicates that the model explains only 30% of the varation of the data, which is not a good fit.

Because we (a) have a lot of data to work with and (b) are seeking to generate a fluid velocisty profile with minimal noise, we are setting a high bar for how well our model explains the varation in the data. The fluid velocity results from any of our linear regressions that return a $R^2$ value < **XX** will be discarded from our analysis. 

In [ ]:
fig, (ax) = plt.subplots(1, 1,figsize=(6,6))
   
ax.scatter(selected_data.frequency_hz, selected_data.speed_mps,
    color = 'none', edgecolors = 'k', marker='o', s = 50, linewidths = 1)

model_y_vals = [] # y values using our model
for n in selected_data.frequency_hz:
    model_y_vals.append(slope * n + intercept)

ax.plot(selected_data.frequency_hz, model_y_vals, 
    color='tab:orange', linestyle='-', linewidth=3, alpha=0.5, label='Linear fit')

ax.scatter(0,intercept,
    color='tab:orange', s = 100)

ax.hlines(0, -40, 40, color = 'k', linewidth = 0.5)
ax.vlines(0, -2, 2, color = 'k', linewidth = 0.5)

ax.set_xlim(-40,40)
ax.set_ylim(-2,2)

ax.set_xlabel('Spinner frequency [hz]')
ax.set_ylabel('Tool speed [mps]')

ax.grid()

## 18.2 Frequency analysis for one flowrate dataframe

Need to consider and refactor this method


In [ ]:
# To do: write error for validation if bottom is less than top. Have it return informative error. 

def analysis_steps(analysis_top, analysis_bottom, step_size):
    '''Make lists that define top and bottom of each analysis interval

    The cross_plot_analysis function requires that we pass in two numbers that 
    define the top and bottom of each data interval that we will do the
    cross-plot analysis on. 

    Args:   analysis_top: The shallowest depth of the spinner analysis interval
            analysis_bottom: The deepest depth of the spinner analysis interval
            step_size: the length of each cross-plot analysis interval within the spinner analysis interval

    Returns:    list_tops: List of top of each cross-plot analysis interval
                list_bots: List of the bottom of each cross-plot analysis interval   
    
    '''

    list_tops = np.arange(
        start = analysis_top, 
        stop = analysis_bottom - step_size, 
        step = step_size)

    list_bots = np.arange(
        start = analysis_top + step_size, 
        stop = analysis_bottom, 
        step = step_size)

    return list_tops, list_bots

# To do: generalise this function so it does not matter what the column headers are

def cross_plot_analysis(dataframe, interval_top, interval_bottom):  
    '''
    Cross plot analysis of spinner frequency and tool speed data to find fluid velocity

    A liner interpolation of spinner frequency (x) and tool speed (y) to find the fluid velocity (y intercept).
    The method also returns the model slope, R squared (goodness of fit), and the number of data points used.

    Args:

    Returns:

    '''  
    temp_df = dataframe[
            (dataframe.depth_m > interval_top) & (dataframe.depth_m < interval_bottom)]
    freq_data = temp_df.frequency_hz.tolist()
    speed_data = temp_df.speed_mps.tolist()
    number_of_df_rows = temp_df.shape[0]
    if number_of_df_rows > 1:
        linear_model = stats.linregress(temp_df.frequency_hz, temp_df.speed_mps)
        r_squared = linear_model[2]**2
    else: 
        linear_model = [None,None]
        r_squared = None
    return freq_data, speed_data, linear_model[1], linear_model[0], r_squared, number_of_df_rows # intercept, slope, rvalue, number of observations

# Need to test for data in the subselection for the linear model. If there is no data then return NaN, if there is data then do the linear fit


In [ ]:
top = 460 # shallowest data depth
bottom = 925 # deepest data depth
step = 0.5 # interval thickness for each cross-plot

# define the interval steps
list_tops, list_bots = analysis_steps(top, bottom, step)

# define our lists
depth = [] # half way between top and bottom of step
frequency_data = []
speed_data = []
fluid_velocity = [] # model intecept
slope = [] # model slope
rsquared = [] # goodenss of fit
obs_num = [] # number of observations in the step

# iterate over our interval steps and calculate a linear fit for each
for top, bot in zip(list_tops, list_bots):
    d = (bot - top)/2 + top 
    depth.append(d)
    fd, sd, v, s, r, obs = cross_plot_analysis(pts_second_rate, top, bot)
    frequency_data.append(fd)
    speed_data.append(sd)
    fluid_velocity.append(v)
    slope.append(s)
    rsquared.append(r)
    obs_num.append(obs) 
    

# To do: write a wrapper function that handles the generation of the final dataframe
# so we can just pass in the dataframe (here pts_second_rate), top, bottom and step

In [ ]:
# Turn the results into a dataframe so we can eaisly explore the data

fluid_velocity_df = pd.DataFrame()
fluid_velocity_df['depth_m'] = depth
fluid_velocity_df['intercept_velocity_mps'] = fluid_velocity
fluid_velocity_df['slope'] = slope
fluid_velocity_df['r_squared'] = rsquared
fluid_velocity_df['obs_num'] = obs_num
fluid_velocity_df['frequency_data_hz'] = frequency_data
fluid_velocity_df['speed_data_mps'] = speed_data
fluid_velocity_df.tail()

### Check the results

In [ ]:
fluid_velocity_df.describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6), sharey=True)

fig.suptitle('Evaluate the Quaility of the Raw Cross-plot Analysis')

ax1.set_title('R-squared value')
im1 = ax1.scatter(fluid_velocity_df.intercept_velocity_mps, fluid_velocity_df.depth_m,
    c = fluid_velocity_df.r_squared, s = 20, linewidths = 0)
fig.colorbar(im1,ax=ax1)

ax2.set_title('Number of data points used')
im2 = ax2.scatter(fluid_velocity_df.intercept_velocity_mps, fluid_velocity_df.depth_m,
    c = fluid_velocity_df.obs_num, s = 20, linewidths = 0)
fig.colorbar(im2,ax=ax2)

ax1.set_ylabel('Depth [m]')
ax1.set_ylim(950,400)

for ax in [ax1,ax2]:
    ax.set_xlabel('Fluid velocity [m/s]')

### Clean the data

Using number of values in the crossplot interval and the R2 value

In [ ]:
# Filter data based on R2 value

filtered_fluid_velocity_df = fluid_velocity_df[(fluid_velocity_df.r_squared > 0.98 )]

# filter data based on number of values

filtered_fluid_velocity_df = filtered_fluid_velocity_df[(filtered_fluid_velocity_df.obs_num > 6 )]


print('before filter =', fluid_velocity_df.shape, 'after filter =', filtered_fluid_velocity_df.shape)

Check the results of cleaning

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6), sharey=True)

fig.suptitle('Evaluate the Quaility of the Cross-plot Analysis after Filtering')

ax1.set_title('R-squared value')
im1 = ax1.scatter(filtered_fluid_velocity_df.intercept_velocity_mps, filtered_fluid_velocity_df.depth_m,
    c = filtered_fluid_velocity_df.r_squared, s = 20, linewidths = 0)
fig.colorbar(im1,ax=ax1)

ax2.set_title('Number of data points used')
im2 = ax2.scatter(filtered_fluid_velocity_df.intercept_velocity_mps, filtered_fluid_velocity_df.depth_m,
    c = filtered_fluid_velocity_df.obs_num, s = 20, linewidths = 0)
fig.colorbar(im2,ax=ax2)

ax1.set_ylabel('Depth [m]')
ax1.set_ylim(950,400)

for ax in [ax1,ax2]:
    ax.set_xlabel('Fluid velocity [m/s]')

## Check a single depth

In [ ]:

def crossplot_check_figure(dataframe, row_number):
    # call the row values from the dataframe
    frequency_data_hz = dataframe.iloc[row_number]['frequency_data_hz'] #.values.tolist()
    speed_data_mps = dataframe.iloc[row_number]['speed_data_mps'] #.values.tolist()
    slope = dataframe.iloc[row_number]['slope']
    intercept = dataframe.iloc[row_number]['intercept_velocity_mps']
    
    # calculate y values using our model
    model_y_vals = []
    for n in frequency_data_hz:
        model_y_vals.append(slope * n + intercept)

    # generate test plot
    fig, (ax) = plt.subplots(1, 1,figsize=(8,8))
    ax.set_title('Format a text string for here that calls R2 and depth and datapoint num') # TO DO
    
    ax.scatter(frequency_data_hz, speed_data_mps,
        color = 'none', edgecolors = 'k', marker='o', s = 50, linewidths = 1)

    ax.plot(frequency_data_hz, model_y_vals, 
        color='tab:orange', linestyle='-', linewidth=3, alpha=0.5, label='Linear fit')

    ax.hlines(0, -40, 40, color = 'k', linewidth = 0.5)
    ax.vlines(0, -2, 2, color = 'k', linewidth = 0.5)

    ax.set_xlim(-40,40)
    ax.set_ylim(-2,2)

    ax.set_xlabel('Spinner frequency [hz]')
    ax.set_ylabel('Tool speed [mps]')

    ax.grid()

    return plt


In [ ]:
crossplot_check_figure(filtered_fluid_velocity_df,0)

# Goal

For at least two fluid injection rates, generate cross-plots to calculate the fluid velocity profile

In [ ]:
# import the first heating temprature log for comparison

heating_0days = pd.read_csv('data-heating0days.csv')
heating_0days.head(2)

You have finished the T21 geothermal well completion test tutoral. Well done!

***

<p><center>© 2021 <a href="https://www.cubicearth.nz/">Irene Wallis</a> and <a href="https://www.linkedin.com/in/katie-mclean-25994315/">Katie McLean</a> <a href="https://creativecommons.org/licenses/by/4.0/"</a></center></p>

<p><center>Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a></center></p>

<p><center>Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.</center></p>

***